In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import os
import pickle

from ml.DataProcessing import *

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.metrics import roc_auc_score,accuracy_score,precision_score,recall_score,f1_score

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 2996 samples in 0.010s...
[t-SNE] Computed neighbors for 2996 samples in 0.363s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2996
[t-SNE] Computed conditional probabilities for sample 2000 / 2996
[t-SNE] Computed conditional probabilities for sample 2996 / 2996
[t-SNE] Mean sigma: 0.328278
[t-SNE] KL divergence after 250 iterations with early exaggeration: 71.369904
[t-SNE] KL divergence after 1000 iterations: 1.331230


c:\Users\bpei\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [2]:
# read the preprocessed data and untrained preprocessor
preprocessDataFolder = 'data/preprocessData'
aggDataDf = pd.read_csv(os.path.join(preprocessDataFolder,'aggDataDf.csv'))
X_train = pd.read_csv(os.path.join(preprocessDataFolder,'X_train.csv'))
y_train = pd.read_csv(os.path.join(preprocessDataFolder,'y_train.csv'))
X_test = pd.read_csv(os.path.join(preprocessDataFolder,'X_test.csv'))
y_test = pd.read_csv(os.path.join(preprocessDataFolder,'y_test.csv'))
preprocessor = pickle.load(open(os.path.join(preprocessDataFolder,'preprocessing.pkl'),'rb'))

c:\Users\bpei\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\bpei\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 0.24.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\bpei\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator ColumnTransformer from version 0.24.2 when using ver

In [3]:
preprocessor

AttributeError: 'ColumnTransformer' object has no attribute 'verbose_feature_names_out'

AttributeError: 'ColumnTransformer' object has no attribute 'verbose_feature_names_out'

AttributeError: 'ColumnTransformer' object has no attribute 'verbose_feature_names_out'

In [2]:
# 不均衡数据集
aggDataDfTest = pd.read_csv('data/studAggregatedData.csv')
delUser=aggDataDfTest[aggDataDfTest.gender=='M'].sample(1000).useraccount_id.values
aggDataDfTestDeled=aggDataDfTest.drop(index=aggDataDfTest[aggDataDfTest.useraccount_id.isin(delUser)].index).reset_index().drop('index',axis=1)
aggDataDfTestDeled.to_csv('data/studAggregatedDataImbalanced.csv',index=False)

data = pd.read_csv('data/dataset.csv')
dataDeled = data.drop(index=data[data.useraccount_id.isin(delUser)].index).reset_index().drop('index',axis=1)
dataDeled.to_csv('data/datasetImbalanced.csv',index=False)

1. 保存数据

In [2]:
# generate the training and testing dataset 保存数据
aggDataDf, X_train, X_test, y_train, y_test, user_train, user_test, preprocessor  = mlDataPrepare(threshold=0.6,attr="gender")
savePreprocessData('data/preprocessData',aggDataDf, X_train, X_test, y_train, y_test, user_train, user_test, preprocessor)

preprocessDataFolder = 'data/preprocessData'
x = SNEProcessor(X_test,preprocessor)
with open(os.path.join(preprocessDataFolder,'x.npy'),'wb') as f:
    np.save(f,x)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 2996 samples in 0.057s...
[t-SNE] Computed neighbors for 2996 samples in 0.496s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2996
[t-SNE] Computed conditional probabilities for sample 2000 / 2996
[t-SNE] Computed conditional probabilities for sample 2996 / 2996
[t-SNE] Mean sigma: 0.328278
[t-SNE] KL divergence after 250 iterations with early exaggeration: 70.619492
[t-SNE] KL divergence after 1000 iterations: 1.291343


In [3]:
# 定义3个模型
modelLR = LogisticRegression()
modelSVC = SVC(kernel='linear',probability=True)
modelDT = DecisionTreeClassifier()
# 生成预测结果
predictorlr, y_pred_proba_lr, y_pred_lr = predict(modelLR, preprocessor, X_train, y_train, X_test)
predictorsvc, y_pred_proba_svc,y_pred_svc = predict(modelSVC, preprocessor, X_train, y_train, X_test)
predictordt, y_pred_proba_dt,y_pred_dt = predict(modelDT, preprocessor, X_train, y_train, X_test)
y_test_performance = np.round(aggDataDf.loc[X_test.index,'performance'].values,3)
x = np.load(open(os.path.join(preprocessDataFolder,'x.npy'),'rb'))
# save lr model and related data
y_scores= predictorlr['classification'].decision_function(X_test)
studDf, studDfTruePass, studDfTrueFail, studDfPredPass, studDfPredFail, studDfFNS, studDfFPS = getStudDf(aggDataDf, X_test, x[:,0], y_test_performance, y_test, y_pred_proba_lr,y_scores, y_pred_lr)
saveData('data/lrModel', predictorlr, studDf, studDfTruePass, studDfTrueFail, studDfPredPass, studDfPredFail, studDfFNS, studDfFPS)

# save svc model and related data
y_scores= predictorsvc['classification'].decision_function(X_test)
studDf, studDfTruePass, studDfTrueFail, studDfPredPass, studDfPredFail, studDfFNS, studDfFPS = getStudDf(aggDataDf, X_test, x[:,0], y_test_performance, y_test, y_pred_proba_svc, y_scores, y_pred_svc)
saveData('data/svcModel', predictorsvc, studDf, studDfTruePass, studDfTrueFail, studDfPredPass, studDfPredFail, studDfFNS, studDfFPS)

# save dt model and related data
y_scores = predictordt['classification'].predict_proba(X_test)[:,1]
studDf, studDfTruePass, studDfTrueFail, studDfPredPass, studDfPredFail, studDfFNS, studDfFPS = getStudDf(aggDataDf, X_test, x[:,0], y_test_performance, y_test, y_pred_proba_dt, y_scores, y_pred_dt)
saveData('data/dtModel', predictordt, studDf, studDfTruePass, studDfTrueFail, studDfPredPass, studDfPredFail, studDfFNS, studDfFPS)

data saved...
data saved...
data saved...


In [8]:
aggDataDf = pd.read_csv('data/dtModel/studDf.csv')
aggDataDf.y_pred_score = aggDataDf.y_pred
aggDataDf.to_csv('data/dtModel/studDf.csv',index=False)